In [650]:
import numpy as np
from scipy.stats import norm

# read file 

data = open("Input/data.txt", "r")
param = open("Input/parameters.txt", "r")



In [651]:
# read lines from file as numbers   

rainfall = []
for line in data:
    rainfall.append(float(line))


N = int(param.readline())

trans_matrix = []

for i in range(N):
    p = [float(x) for x in param.readline().split()]
    trans_matrix.append(p)


# read distro

means = [float(x) for x in param.readline().split()]
sds = [np.sqrt(float(x)) for x in param.readline().split()]





In [652]:
# gaussian distribution function

def gauss(x , mean , sd):
    prob_density = norm.pdf(x, loc=mean, scale=sd)
    return prob_density


# find stationary distribution for given transition matrix

def get_stationary_distribution(A,pi):
    A = np.array(A)
    pi = np.array(pi)
    
    N = len(A)
    
    # initial state
    x = pi
    
    # recursion
    for i in range(N):
        x = np.dot(x,A)
    
    return x


In [653]:

# emission probabilities
emisson_matrix = []

n = len(trans_matrix)

for i in range(n):
    prob = []
    for d in rainfall:
        prob.append(gauss(d, means[i], sds[i]))
    emisson_matrix.append(prob)



# viterbi algorithm

In [654]:

def viterbi(A,B,pi) :
    A = np.array(A)
    B = np.array(B)
    pi = np.array(pi)
 
    N = len(A)
    T = len(B[0])

    # initial state 
    delta = np.zeros((N,T))
    psi = np.zeros((N,T))
    delta[:,0] = np.log( pi*B[:,0] )
    psi[:,0] = 0
    
    # recursion
    for t in range(1,T):
        for j in range(N):
            # log of max prob
            max_prob = -np.inf
            max_state = 0

            for i in range(N):
                prob = delta[i,t-1] + np.log( A[i,j] * B[j,t] )
                if prob > max_prob:
                    max_prob = prob
                    max_state = i
            
            delta[j,t] = max_prob
            psi[j,t] = max_state

    
    # backtracking
    
    sol = np.zeros(T)
    sol[T-1] = np.argmax(delta[:,T-1])
    
    for t in range(T-1,0,-1):
        sol[t-1] = psi[int(sol[t]),t]
    
    return sol



pi = get_stationary_distribution(trans_matrix,[0.5,0.5])

Sol =  viterbi(trans_matrix,emisson_matrix,pi)


# solution write in output file

out = open("my_output/wo_learing.txt","w")

for s in Sol :
    if s == 1 :
        out.write("La Nina\n")
    else:
        out.write("El Nino\n")



    

# Baum Welch Algorithm

In [655]:
# baum welch algorithm for learning parameters

def Baum_welch(A,B,pi) :
    A = np.array(A)
    B = np.array(B)
    pi = np.array(pi)

    N = len(A)
    T = len(B[0])

    # forward 
    alpha = np.zeros((N,T))
    alpha[:,0] = pi*B[:,0]

    for t in range(1,T):
        for j in range(N):
            alpha[j,t] = np.sum(alpha[:,t-1]*A[:,j])*B[j,t]

    # normalize alpha 
    for i in range(N):
        alpha[i,:] /= np.sum(alpha[i,:])
    
    


In [656]:
Baum_welch(trans_matrix,emisson_matrix,pi)

alpha :  [[9.31459231e-138 1.75048183e-166 7.71220095e-155 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [9.01419844e-001 8.85081756e-002 9.34972731e-003 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]]
